In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_raw = pd.read_csv('ACI-IoT-2023.csv')

In [4]:
df_raw.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Connection Type
0,192.168.1.7-54.230.163.60-33344-443-6,192.168.1.7,33344,54.230.163.60,443,6,2023-01-11 09:43:40,379933,11,11,...,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired
1,192.168.3.6-91.189.91.48-36754-80-6,192.168.3.6,36754,91.189.91.48,80,6,2023-01-11 09:43:51,205637,3,3,...,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired
2,192.168.3.6-91.189.91.48-36754-80-6,192.168.3.6,36754,91.189.91.48,80,6,2023-01-11 09:43:51,0,2,0,...,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired
3,192.168.1.9-192.168.1.1-8080-40426-6,192.168.1.9,8080,192.168.1.1,40426,6,2023-01-11 09:43:49,5030379,1,2,...,0.0,0.0,0.0,0.0,8.494231e+14,1.201266e+15,1.698846e+15,5.030379e+06,Benign,wired
4,192.168.1.20-35.232.111.17-40054-80-6,192.168.1.20,40054,35.232.111.17,80,6,2023-01-11 09:43:56,72278,3,4,...,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,Benign,wired


In [5]:
df_raw.shape

(1231411, 85)

In [6]:
df_raw.columns

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Len

In [7]:
pd.set_option('display.max_rows', None)
df_raw.columns[df_raw.dtypes=='object'].to_list()

['Flow ID', 'Src IP', 'Dst IP', 'Timestamp', 'Label', 'Connection Type']

In [8]:
df_raw.dtypes['Protocol']

dtype('int64')

In [9]:
label_counts = df_raw['Label'].value_counts()
label_counts

Label
Port Scan             441282
Benign                329295
ICMP Flood            225234
Ping Sweep             71928
DNS Flood              46935
Vulnerability Scan     39537
OS Scan                37524
Slowloris              18643
SYN Flood              13857
Dictionary Attack       6380
UDP Flood                791
ARP Spoofing               5
Name: count, dtype: int64

In [10]:
type_counts = df_raw['Connection Type'].value_counts()
type_counts

Connection Type
wireless    742758
wired       488653
Name: count, dtype: int64

# Data preprocessing

In [11]:
remove_cols = ['Flow ID', 'Src IP', 'Dst IP', 'Timestamp', 'Label']

In [12]:
X = df_raw.drop(columns=remove_cols)
y = df_raw['Label']

In [13]:
X.head()

,Src Port,Dst Port,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Connection Type
0,33344,443,6,379933,11,11,720.0,6169.0,517.0,0.0,...,32,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
1,36754,80,6,205637,3,3,87.0,189.0,87.0,0.0,...,32,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
2,36754,80,6,0,2,0,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
3,8080,40426,6,5030379,1,2,6.0,0.0,6.0,6.0,...,20,0.0,0.0,0.0,0.0,8.494231e+14,1.201266e+15,1.698846e+15,5.030379e+06,wired
4,40054,80,6,72278,3,4,87.0,148.0,87.0,0.0,...,32,0.0,0.0,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired


In [14]:
y.head()

0    Benign
1    Benign
2    Benign
3    Benign
4    Benign
Name: Label, dtype: object

### Encode Columns

In [15]:
y = pd.get_dummies(y, dtype=int)
y.head()

,ARP Spoofing,Benign,DNS Flood,Dictionary Attack,ICMP Flood,OS Scan,Ping Sweep,Port Scan,SYN Flood,Slowloris,UDP Flood,Vulnerability Scan
0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0
